## Cleaning Process 
Nesta etapa vamos estar realizando a limpeza do dataset na seguinte ordem:

1. Identificação dos dados faltantes
2. Valores inválidos 
3. Consistência entre colunas 
4. Duplicatas e Datas



In [2]:
# Importação das Bibliotecas
import pandas as pd  
import numpy as np

### 1.  Identificação de dados faltantes

In [5]:
# Importando o dataset 
df = pd.read_csv("dirty_cafe_sales.csv")

# Contar valores ausentes e Inválidos
missing_stats = df.apply(lambda x: x.isin(['UNKNOWN', 'ERROR', 'None', np.nan]).sum())
print('Valores Problemáticos por Coluna:')
print(missing_stats)


Valores Problemáticos por Coluna:
Transaction ID         0
Item                 969
Quantity             479
Price Per Unit       533
Total Spent          502
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64


In [6]:
# Solução para Item e Payment Method 
df['Item'] = df['Item'].replace(['UNKNOWN', 'ERROR', 'None'], np.nan)
df['Payment Method'] = df['Payment Method'].replace(['UNKNOWN', 'ERROR', 'None'], np.nan)

### 2. Valores inválidos

In [8]:
# Identificar valores não numéricos
invalid_quantity = pd.to_numeric(df['Quantity'], errors='coerce').isna().sum()
print(f"Quantidades Inválidas: {invalid_quantity} registros")

# Substituir por 1 (suposição conservadora) ou remover
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce').fillna(1)

Quantidades Inválidas: 479 registros


In [9]:
# Correção de preços
price_map = {
    'Coffee' : 2, 'Tea': 1.5, 'Sandwich': 4, 'Salad': 5,
    'Cake': 3, 'Cookie': 1, 'Smoothie': 4, 'Juice': 3
}

# Corrigir os preços usando item por refrência
df['Price Per Unit'] = df.apply(
    lambda row: price_map[row['Item']] if (row['Item'] in price_map) else row['Price Per Unit'],
    axis=1
)

### 3. Consistência entre colunas


In [16]:
# Substituir valores inválidos e converter para float
df['Total Spent'] = df['Total Spent'].replace(['ERROR', 'UNKNOWN', 'None'], np.nan)
df['Total Spent'] = (
    pd.to_numeric(df['Total Spent'].astype(str).str.replace(',', '.'), errors='coerce')
)

# Preencher valores ausentes com Quantity * Price Per Unit
df['Total Spent'] = df['Total Spent'].fillna(df['Quantity'] * df['Price Per Unit'])

# Remover linhas com Total Spent ausente
df = df.dropna(subset=['Total Spent'])

In [14]:
# Substituir variações de texto
df['Location'] = df['Location'].str.replace(r'\bIn[-\s]?[Ss]tore\b', 'In-store', regex=True)
df['Location'] = df['Location'].str.replace('Takeaway', 'Takeout')

### 4. Duplicatas e Datas

In [18]:
# verificar duplicatas
duplicates = df.duplicated(subset=['Transaction ID'], keep=False)
print(f'Transsações duplicadas: {duplicates.sum()} registros')

# Remover duplicatas (mantendo a primeira ocorrência)
df = df.drop_duplicates(subset=['Transaction ID'], keep='first')

Transsações duplicadas: 0 registros


In [21]:
# Converter para datetime e identificar inválidos
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce')
invalid_dates = df['Transaction Date'].isna().sum()

# Remover registros com datas inválidas
df = df.dropna(subset=['Transaction Date'])

### Diversas colunas

In [23]:
# Remover linhas onde Item está ausente (já que Price Per Unit também estará ausente)
df = df.dropna(subset=['Item', 'Price Per Unit'])

print(f"Registros restantes após remoção: {len(df)}")

moda_pagamento = df['Payment Method'].mode()[0]  # Ex: 'Credit Card'
df['Payment Method'] = df['Payment Method'].fillna(moda_pagamento)

print(f"Método de pagamento mais comum: {moda_pagamento}")

df['Location'] = df['Location'].fillna('Unknown')

print("Valores ausentes por coluna após limpeza final:")
print(df.isnull().sum())

Registros restantes após remoção: 8613
Método de pagamento mais comum: Digital Wallet
Valores ausentes por coluna após limpeza final:
Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64


In [24]:
# Salvando Dataset Tratado
df.to_csv('clean_cafe_sales.csv', index=False, encoding='utf-8')